# Three-body **Fermion** bound state problem

In this notebook, we modify the lecture notebook so that it accomodates bound state problems for fermions instead. This is equivalent to modifying the permutation operator to include the permutations due to spins as well as the permutations between spins and orbital quantum numbers.


Code version 3:
- Cleaned up previous version


In [1]:
# import all classes that we will use here
# all classes that need not be modified is stored here
from tbs_cls import OBEpot, TwoBody, Cubherm

import numpy as np
import matplotlib.pyplot as plt
import math as m

In [2]:
# parameters for OBE interaction
parasets=[[300.0, -0.09827953494014054],
 [400.0, -0.028203145146196713],
 [500.0, -0.0004221894040945335],
 [600.0, 0.012857431330421717],
 [700.0, 0.020167185806378923],
 [800.0, 0.024707945457255083],
 [900.0, 0.027865200396659445],
 [1000.0, 0.030308007813785776],
 [1100.0, 0.03239034331482156],
 [1200.0, 0.03431611357447293]]

In [3]:
# calculate binding energies

# go through parameters of fit
for para in parasets:
    # determine interacion and set up solver 
    pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])
    solver=TwoBody(pot=pot,np1=40,np2=20,pa=1.0,pb=7.0,pc=35.0,mred=938.92/2,l=0,
                            nr1=40, nr2=20, ra=1.0, rb=5.0, rc=20.0, 
                            np1four=400,np2four=200)
    
    # perform energy search for this parameter set
    ener,lam,pmom,wf=solver.esearch(neigv=1,e1=-2.0/TwoBody.hbarc,e2=-2.5/TwoBody.hbarc)
    
    # get Fourier trafo (together with r grid points)
    rp,wfr=solver.fourier(wf)
    # calculate norm and rms radius (deviation of the norm from 1 gives idea of accuracy)
    norm,rms=solver.rms(wfr)
    
    # print energy, eigenvalue (should be one), norm and 1/2 rms (= radius to the CM for equal mass particles)
    print("{0:15.6e}   {1:15.6e}  {2:15.6e}   {3:15.6e}".format(ener*TwoBody.hbarc,lam, norm, rms/2.0)) #rms is printed for distance to CM assuming equal masses

    
# for some timing statistics print measurements of last runs 

print("Preparatione time: {0:15.6e} sec".format(solver.preptime))
print("Energy search time: {0:15.6e} sec".format(solver.runtime))  


  -2.224997e+00      1.000000e+00  9.997794e-01+0.000000e+00j   2.096766e+00+0.000000e+00j
  -2.224993e+00      1.000000e+00  9.997841e-01+0.000000e+00j   2.069329e+00+0.000000e+00j
  -2.224990e+00      1.000000e+00  9.997849e-01+0.000000e+00j   2.064349e+00+0.000000e+00j
  -2.224991e+00      1.000000e+00  9.997849e-01+0.000000e+00j   2.064681e+00+0.000000e+00j
  -2.224992e+00      1.000000e+00  9.997846e-01+0.000000e+00j   2.066091e+00+0.000000e+00j
  -2.224992e+00      1.000000e+00  9.997844e-01+0.000000e+00j   2.067504e+00+0.000000e+00j
  -2.224993e+00      1.000000e+00     9.997843e-01      2.068679e+00
  -2.224994e+00      1.000000e+00  9.997842e-01+0.000000e+00j   2.069602e+00+0.000000e+00j
  -2.224994e+00      1.000000e+00  9.997841e-01+0.000000e+00j   2.070315e+00+0.000000e+00j
  -2.224991e+00      1.000000e+00  9.997840e-01+0.000000e+00j   2.070864e+00+0.000000e+00j
Preparatione time:    1.352133e+00 sec
Energy search time:    1.252850e-02 sec


## Two-Body Off-shell t-matrix

Here the three-body t-matrix (off-shell) is defined that solves the LS equation for thr three-body problem.
- This includes spin, total angular momentum, and isospin state of the two-body system.

The following is modified from the original code:
- argument `jmax` is included, to quantify the max total ang momentum for the two-body state
- we define `smax` and `tmax` for the max spin and isospin state.
    - for `tmax`, this is always set to 1 since by addition of isospin states, t=0,1 is both possible
    - note that this is also true for the spin states (i.e. s=0,1 both possible)
- we define `alpha_list` much like when dealing with the three-body bound state
    - contains dictionary of `l, s, j, t` values
    - then the potential matrix construction depends on `alpha` and not `l` now
    - same thing with the t-matrix, instead of `l` being an index, each `alpha` becomes an index instead (in `_prep_tmat()`)

Once finalized this will be put into the classes .py file.


In [4]:
# next extend the class for twobody to scattering 
from scipy.special import legendre
import timeit 

class TwoBodyTMat(TwoBody):
    """This class defines the off-shell t-matrix for a three-body bound state.
    
       The class assumes three identical particles. It also contains the 
       initialization of grid points for the three-body problem. 
       
       Here we modify it for fermionic case. We add the following:
       - j12max, parameter for max total angular momentum
       - take into account 
    """
    
    def __init__(self, pot, np1=20, np2=10, pa=1.0, pb=5.0, pc=20.0, 
                            nq1=20, nq2=10, qa=1.0, qb=5.0, qc=20.0, 
                            mass=938.92,lmax=0,
                            nr1=20, nr2=10, ra=1.0, rb=5.0, rc=20.0, 
                            nrho1=20, nrho2=10, rhoa=1.0, rhob=5.0, rhoc=20.0, 
                            np1four=200,np2four=100):
        """Initialization of grid points and interaction for the solution of the three-body problem. 
        
           Parameter: 
           pot -- object that defines the potential matrix elements (e.g. of class OBEpot).
           np1 -- number of p grid points in interval [0,pb] 
           np2 -- number of p grid points in interval [pb,pc]
           pa  -- half of np1 points are in interval [0,pa]
           pb  -- interval boundary as defined above 
           pc  -- upper integration boundary for the solution of the integral equation 
           nq1 -- number of q grid points in interval [0,qb] 
           nq2 -- number of q grid points in interval [qb,qc]
           qa  -- half of np1 points are in interval [0,qa]
           qb  -- interval boundary as defined above 
           qc  -- upper integration boundary for the solution of the integral equation 
           mass -- particle mass of the three identical bosons in MeV
           
           nr1 -- number of r (related to p) points in interval [0,rb] 
           nr2 -- number of r (related to p) points in interval [rb,rc]
           ra  -- half of np1 points are in interval [0,ra]
           rb  -- interval boundary as defined above 
           rc  -- upper integration boundary for the solution of the integral equation 
           nrho1 -- number of rho (related to q) points in interval [0,rhob] 
           nrho2 -- number of rho (related to q) points in interval [rhob,rhoc]
           rhoa  -- half of np1 points are in interval [0,rhoa]
           rhob  -- interval boundary as defined above 
           rhoc  -- upper integration boundary for the solution of the integral equation 
           
           np1four -- number of p or q  points in interval [0,pb] or[0,qb]   for Fourier trafo
           np2four -- number of p or q points in interval [pb,pc] or [qb,qc] for Fourier trafo
           
           lmax  -- maximal two-body angular momentum to be taken into account
           jmax  -- maximal two-body total angular momentum
           """
        
        
        # first use the TwoBody class to keep the main parameters 
        super().__init__(pot,np1,np2,pa,pb,pc,mass/2,0,nr1,nr2,ra,rb,rc,np1four,np2four)

        self.nq1 = nq1
        self.nq2 = nq2
        self.nqpoints  = nq1+nq2 
        self.mass=mass/self.hbarc
        self.qa=qa
        self.qb=qb
        self.qc=qc
        self.lmax=lmax 
        # self.jmax = jmax
        self.smax= 1
        self.jmax = lmax + self.smax
        self.tmax = 1

        self.nrho1 = nrho1
        self.nrho2 = nrho2
        self.nrhopoints  = nrho1+nrho2 
        self.rhoa=rhoa
        self.rhob=rhob
        self.rhoc=rhoc


        # store grid points and weights for integral equations
        self.qgrid,self.qweight = self._trns(self.nq1,self.nq2,self.qa,self.qb,self.qc)
 
        # store grid points and weights for r space wave functions
        self.rhogrid,self.rhoweight = self._trns(self.nrho1,self.nrho2,self.rhoa,self.rhob,self.rhoc)
        
        # store grid points and weights for Fourier trafo 
        self.qfourgrid,self.qfourweight = self._trns(self.np1four,self.np2four,self.qa,self.qb,self.qc)

        # create alpha matrix for reduced storage

        self.alpha_list = []
        m=0
        for l in range(self.lmax+1):
          for s in range(self.smax+1):
            for j in range(self.smax+1):
              for t in range(self.tmax+1):
                self.alpha_list.append({"m":m, "l":l, "s":s, "j":j, "t":t})
                m+=1

        self.nalpha = len(self.alpha_list)
        # and prepare actual potential matrix elements for all angular momenta
        self.vmatpw=np.empty((self.npoints,self.npoints,self.nalpha),dtype=np.double)
        for m, alpha in enumerate(self.alpha_list):
          l = alpha["l"]
          for i in range(self.npoints):
              for k in range(self.npoints):
                  if l==0:
                    self.vmatpw[i,k,m]=self.vmat[i,k]  
                  else:    
                    self.vmatpw[i,k,m]=self.pot.v(self.pgrid[i],self.pgrid[k],l)
        
        self.tmattime=0.0
        
# now turn to scattering and solve for LS equation to get tmatrix (on- and offshell)
    def prep_tmat(self,E):
      """Prepares all necessary t-matrix elements up to l=lmax.
      
         Starts the calculation of the t-matrix for a given three-body energy. 
      """  
      self.tmattime-=timeit.default_timer() 
      # prepare off-shell energies for t-matrix 
      etmat=E-0.75*self.qgrid**2/self.mass   # note that this is a negative energy < E_b(two-body) 
             
      # prepare numpy array that keeps all tmatrix elements 
      tmat=np.empty((self.nalpha,self.nqpoints,self.npoints,self.npoints),dtype=np.double)
      
      for m, _ in enumerate(self.alpha_list):

        for ie in range(self.nqpoints):
          # to deal with delta function
          amat=np.identity(self.npoints,dtype=np.double)
          # now add the second part of the definition of the matrix   
          # below is naive version, go back to this if need to debug
          # for i in range(self.npoints):
          #     for k in range(self.npoints):
          #         amat[i,k]+=-self.vmatpw[i,k,m]*self.pgrid[k]**2 \
          #                       /(etmat[ie]-self.pgrid[k]**2/(2*self.mred))*self.pweight[k]  

          amat-=np.einsum("ij...,j->ij...",self.vmatpw[:,:,m],self.pgrid[:]**2 \
                                /(etmat[ie]-self.pgrid[:]**2/(2*self.mred))*self.pweight[:])
          bmat=self.vmatpw[:,:,m]

          # finally solve set of equations and store in complete array 
          tmat[m,ie,:,:]=np.linalg.solve(amat,bmat)
        
      self.tmattime+=timeit.default_timer() 
      # return offshell matrix for all energies and angular momenta   
      return tmat
            


In [5]:
pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])
solver = TwoBodyTMat(pot, lmax=0)
tmat = solver.prep_tmat(-19)
print("Time for computing t-matrix: ", solver.tmattime)

Time for computing t-matrix:  0.351440199999999


## Three-body Bound Problem

Here we compute the three-body bound problem.

### What we modified
- included arguments `l3max`, `bj`
    - now we set J and evaluate (possible combinations of) L, S from this 
- evaluate qnalpha including `l, lam, s, s3, j, j3, t, t3, bj, bt`
    - this means indexing over alpha implies indexing over all such coefficients
- create a separate list that contains possible `bl, bs` from given `bj`
- in `_prep_pmat()`:
    - change indices to allow for `s, j, t` values in the spherical harmonics
    - change indices for clebsch-gordon coefficients for coupled sph harm 
    - changes indices for coupled sph harm
        - here the matrix now indexes for each alpha rather than each `s,j,t` values
    - construct new permutation operator
        - include Wigner-9j symbols for orbital ang, spin, total ang (and prefactors)
        - include Wigner-6j symbols for spin and isospin parts (and prefactors)
        - sum over all possible `bl, bs` values
    - for spline interpolation, this is the same as both old and new implementations use
      `alpha` as the index.
- in `_prep_faddeev`: change indexing for Faddeev eq to have indexing for `alpha` instead of `l`.

In [6]:
# definition of a ThreeBody class for the calculation of bound states
from numpy.polynomial.legendre import leggauss
from scipy.special import sph_harm
from sympy.physics.quantum.cg import CG, Wigner9j, Wigner6j


class ThreeBody(TwoBodyTMat):
    """Provides routines for the implementation of the permutation operator and application of the bound state kernel."""
        
    def __init__(self, pot, np1=20, np2=10, pa=1.0, pb=5.0, pc=20.0, 
                            nq1=20, nq2=10, qa=1.0, qb=5.0, qc=20.0, 
                            nx=12,
                            mass=938.92,lmax=0, l3max=0, bj=0.5,
                            nr1=20, nr2=10, ra=1.0, rb=5.0, rc=20.0, 
                            nrho1=20, nrho2=10, rhoa=1.0, rhob=5.0, rhoc=20.0, 
                            np1four=200,np2four=100):     
        """Initializes the permutation operator for the three-body calculation and prepares application of Faddeev kernel.
        
           Parameters: 
           
           pot -- object that defines the potential matrix elements (e.g. of class OBEpot).
           np1 -- number of p grid points in interval [0,pb] 
           np2 -- number of p grid points in interval [pb,pc]
           pa  -- half of np1 points are in interval [0,pa]
           pb  -- interval boundary as defined above 
           pc  -- upper integration boundary for the solution of the integral equation 
           nq1 -- number of q grid points in interval [0,qb] 
           nq2 -- number of q grid points in interval [qb,qc]
           qa  -- half of np1 points are in interval [0,qa]
           qb  -- interval boundary as defined above 
           qc  -- upper integration boundary for the solution of the integral equation 
           
           nx -- angular grid points for the permutation operator
           
           mass -- particle mass of the three identical bosons in MeV
           
           nr1 -- number of r (related to p) points in interval [0,rb] 
           nr2 -- number of r (related to p) points in interval [rb,rc]
           ra  -- half of np1 points are in interval [0,ra]
           rb  -- interval boundary as defined above 
           rc  -- upper integration boundary for the solution of the integral equation 
           nrho1 -- number of rho (related to q) points in interval [0,rhob] 
           nrho2 -- number of rho (related to q) points in interval [rhob,rhoc]
           rhoa  -- half of np1 points are in interval [0,rhoa]
           rhob  -- interval boundary as defined above 
           rhoc  -- upper integration boundary for the solution of the integral equation 
           
           np1four -- number of p or q  points in interval [0,pb] or[0,qb]   for Fourier trafo
           np2four -- number of p or q points in interval [pb,pc] or [qb,qc] for Fourier trafo
           
           lmax  -- maximal two-body angular momentum to be taken into account    
           bl    -- total orbital angular momentum L ("big l")           
           jmax  --- maxilamal two-body total angular momentum
           l3max -- maximal one-body orbital angular momentum
           bs  -- total spin S, always set to 1.5
        """
        
        # first initialize the tmatrix class (do not calc the tmatrix yet)
        super().__init__(pot,np1,np2,pa,pb,pc,nq1,nq2,qa,qb,qc,
                         mass,lmax,
                         nr1,nr2,ra,rb,rc,nrho1,nrho2,rhoa,rhob,rhoc,
                         np1four,np2four)
        
        # prepare angular grid points for the permutation operator 
        self.nx=nx
        self.xp,self.xw = leggauss(nx)
        
        # prepare partial wave channels for total angular momentum bl
        # parameters for two-body system already defined in TwoBodyTMat
        
        # 3rd particle parameters
        # smax, jmax, tmax are all defined in TwoBodyTmat object
        self.lammax = l3max   # max for orbital angular momentum for 3rd particle
        self.s3 = 0.5   # always fixed to be spin-1/2 particles
        self.t3 = 0.5   # isospin state is always 1/2 for proton / neutron

        # total parameters
        self.j3max = self.lammax + self.s3
        self.bj = bj
        self.blmax = int(bj-0.5)
        self.bsmax = bj
        
        
        # now construct pertial wave channels, including:
        # l, l3, bl, s, s3, bs, j, j3, bj, t, t3, bt
        self.qnalpha=[]
        alpha=0

        start = timeit.default_timer()

        for l in range(lmax+1):
            for s in range(self.smax+1):
                for j in range(self.jmax+1):
                  for t in range(self.tmax+1):
                    if(l%2==0):   # take only symmetric pw (Pauli)  
                      # for lam in range(abs(l-bl),l+bl+1):
                      for lam in range(self.lammax+1):
                        for j3 in np.arange(abs(j-self.bj),j+self.bj+1):
                          self.qnalpha.append({"alpha":alpha, 
                                                "l":l,"lam":lam,
                                                "s":s, "s3":self.s3,
                                                "j":j, "j3":j3, "bj":self.bj,
                                                "t":t, "t3":self.t3, "bt":1.5
                                              })
                          alpha+=1


        # array for bl, bs
        self.blbs_list = []
        for bl in range(self.blmax+1):
          bs = bj-bl
          self.blbs_list.append({"bl":bl, "bs":bs})

        self.nalpha=len(self.qnalpha)
        end = timeit.default_timer()
        print("Duration of alpha list construction: ", end-start)

        

        # split time measurements in several pieces to find relevant loops 
        self.timepermangle=0
        self.timeylam=0
        self.timeyl=0
        self.timeystarl=0
        self.timeclebsch=0
        self.timeylylam=0
        self.timegcalc=0
        self.timespl=0
        self.timepmat=0
        
        self.gpreptime=-timeit.default_timer()        
        # this prepares the G function and splines to be used for the preparation of the 
        # kernel later (pmat = permutation matrix)
        self.pmat=self._prep_perm()
        self.gpreptime+=timeit.default_timer()

        self.fadpreptime=0
        self.fadsolvetime=0
        self.numiter=0

        
    def _angle(self,px,py,pz):
        """Auxiliary routine to determine magnitude, phi, and theta of three component vector. 
        
           Parameters:
           px,py,pz -- cartesian components of a vector 
           
           returns magntitude, theta and phi angles.
        """
    
        pmag=np.sqrt(px**2+py**2+pz**2)
        theta=np.where(pmag!=0.0,np.arccos(pz/pmag),0.0)
             
        phi=theta # copy shape of theta to phi 
        phi=1.5*m.pi # and set to constant value
        
        # prepare bool arrays for px,py > 0 < 0  with shape of phi 
        
        pxgt0=(phi==0)  # copy shape
        pxgt0=(px>0)    # test 

        pxlt0=(phi==0)  # copy shape
        pxlt0=(px<0)    # test 

        pxeq0=(phi==0)  # copy shape
        pxeq0=(px==0)   # test 

        pygt0=(phi==0)  # copy shape
        pygt0=(py>0)    # test 
                      
        np.where(pxgt0 & pygt0,np.arctan(py/px),phi)
        np.where(pxgt0 & np.invert(pxgt0),2*m.pi-np.arctan(-py/px),phi)
        np.where(pxlt0 & pygt0,m.pi-np.arctan(-py/px),phi)
        np.where(pxlt0 & np.invert(pygt0),m.pi+np.arctan(py/px),phi)
        np.where(pxeq0 & pygt0,0.5*m.pi,phi)
            
        return pmag,theta,phi     
    
    
    def _lmindx(self,l,m):
        """Combined unique index for l and m.
        
           Nice trick: since quantum numbers lm are linked to each other, this combined 
           index allows one to store the results depending on lm using the memory more efficiently. 
        """        
        return l**2+l+m
      
    # KW: here the permutation operator is constructed
    # KW: we need to modify here to include spin CG coefficients
    def _prep_perm(self):
        """Prepares and return an array for the application of the permutation operator.
        
           The matrix is based on G_{alpha,alphap}(q,qp,x) and is combined to be 
           directly applicable to be summed  with tmatrix.
        """
        self.timepermangle-=timeit.default_timer()
        # prepare shifted momenta and angles for the symmetric permutation 
        pip=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)        
        pi=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)        
        
        thetap=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)
        theta=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)
        thetapp=np.empty((self.nx),dtype=np.double)
        
        for ix in range(self.nx):
          xval=self.xp[ix] 
          thetapp[ix]=np.arccos(xval)
          for jq in range(self.nqpoints):
            qpval=self.qgrid[jq]
            for iq in range(self.nqpoints):
              qval=self.qgrid[iq]
            
              px=qpval*np.sqrt(1.0-xval**2)
              py=0.0
              pz=0.5*qval+qpval*xval 
              pi[iq,jq,ix],theta[iq,jq,ix],phi=self._angle(px,py,pz)
                
              px=-0.5*qpval*np.sqrt(1.0-xval**2)
              py=0.0
              pz=-qval-0.5*qpval*xval 
              pip[iq,jq,ix],thetap[iq,jq,ix],phi=self._angle(px,py,pz)

        self.timepermangle+=timeit.default_timer()

        # prepare spherical harmonics and store based on lmindx 
        # number of lam,mu und l,mu combinations 
        nlamindx=self._lmindx(self.lammax,self.lammax)+1
        nlindx=self._lmindx(self.lmax,self.lmax)+1
        
        self.timeylam-=timeit.default_timer()
        # array for Y_{lam mu}(hat qp) (real is sufficient since phi=0)
        ylam=np.empty((nlamindx,self.smax+1, self.jmax+1,self.tmax+1, self.nx),dtype=np.double)
        for lam in range(self.lammax+1):
          for mu in range(-lam,lam+1):
            ylam[self._lmindx(lam,mu),:,:,:,:]=np.real(sph_harm(mu,lam, 0, thetapp))
        self.timeylam+=timeit.default_timer()
        
        self.timeyl-=timeit.default_timer()
        # array for Y_{l mu}(-q-0.5qp) (real is sufficient since phi=0)
        yl=np.empty((nlindx,self.smax+1, self.jmax+1,self.tmax+1,self.nqpoints,self.nqpoints,self.nx),dtype=np.double)
        for l in range(self.lmax+1):
          for mu in range(-l,l+1):
            yl[self._lmindx(l,mu),:,:,:,:,:,:]=np.real(sph_harm(mu,l, 0, thetap))
        self.timeyl+=timeit.default_timer()

        self.timeystarl-=timeit.default_timer()
        # array for Y*_{l mu}(0.5q+qp) (real is sufficient since phi=0)
        ystarl=np.empty((nlindx,self.smax+1, self.jmax+1,self.tmax+1,self.nqpoints,self.nqpoints,self.nx),dtype=np.double)
        for l in range(self.lmax+1):
          for mu in range(-l,l+1):
            ystarl[self._lmindx(l,mu),:,:,:,:,:,:]=np.real(sph_harm(mu,l, 0, theta))
        self.timeystarl+=timeit.default_timer()

        # now prepare the necessary Clebsch-Gordan coefficients
        # we need (l lam L, M 0 M)  and (l lam L,mu M-mu,M)
        # I assume that L is smaller than the lmax or lammax therefore M=-L,L
        # the smallest index for storage 
        self.timeclebsch-=timeit.default_timer()

        cg=np.zeros((self.nalpha,2*self.blmax+1),dtype=np.double)
        cgp=np.zeros((self.nalpha,2*self.lmax+1,2*self.blmax+1),dtype=np.double)

        for blbs in self.blbs_list:
          bl = blbs["bl"]
          for qnset in self.qnalpha:
            alpha=qnset["alpha"]

          # make array for each bl over here
          # cg_bl = np.zeros(2*bl+1,dtype=np.double)
          # cgp_bl = np.zeros((2*self.lmax+1, 2*bl+1),dtype=np.double)
            for bm in range(-bl,bl+1):
              # cg_bl[bm+bl]=float(CG(qnset["l"],bm,qnset["lam"],0,bl,bm).doit())
              cg[qnset["alpha"],bm+bl]=float(CG(qnset["l"],bm,qnset["lam"],0,bl,bm).doit())
              for mu in range(-qnset["l"],qnset["l"]+1):
                # cgp_bl[mu+qnset["l"],bm+bl] = float(CG(qnset["l"],mu,qnset["lam"],bm-mu,bl,bm).doit())
                cgp[qnset["alpha"],mu+qnset["l"],bm+bl]=float(CG(qnset["l"],mu,qnset["lam"],bm-mu,bl,bm).doit())
        
        self.timeclebsch+=timeit.default_timer()

        # now we can perform the mu summation for the combination of coupled spherical harmonics 
        self.timeylylam-=timeit.default_timer()

        ylylam=np.zeros((self.nalpha,2*self.blmax+1,self.nqpoints,self.nqpoints,self.nx),dtype=np.double)
        for blbs in self.blbs_list:
          bl=blbs["bl"]
          for qnset in self.qnalpha:  # go through allowed l,lam combinations
              alphap=qnset["alpha"]
              l=qnset["l"]
              lam=qnset["lam"]
              s=qnset["s"]
              s3 = qnset["s3"]
              j=qnset["j"]
              j3=qnset["j3"]
              t=qnset["t"]
              t3=qnset["t3"]
              # bl=qnset["bl"]
              # print(l,s,j,t)
              for bm in range(-bl,bl+1):
                  for mu in range(-l,l+1):
                    lmindx=self._lmindx(l,mu)
                    if abs(bm-mu)<=lam:
                      lamindx=self._lmindx(lam,bm-mu)
                      ylylam[alphap,bm+bl,:,:,:]+=cgp[alphap,mu+l,bm+bl]*yl[lmindx,s,j,t,:,:,:]*ylam[lamindx,s,j,t,:]
        
        self.timeylylam+=timeit.default_timer()

        self.timegcalc-=timeit.default_timer()

        gfunc_bl=np.zeros((self.nalpha,self.nalpha,self.blmax+1,self.nqpoints,self.nqpoints,self.nx),dtype=np.double)

        for blbs in self.blbs_list:
          bl = blbs["bl"]
          bs = blbs["bs"]
          for qnset in self.qnalpha:  # go through allowed l,lam combinations


              alpha=qnset["alpha"]
              l=qnset["l"]
              lam=qnset["lam"]
              s=qnset["s"]
              s3 = qnset["s3"]
              j=qnset["j"]
              j3=qnset["j3"]
              t=qnset["t"]
              t3=qnset["t3"]
              bj=qnset["bj"]
              bt=qnset["bt"]

              for qnsetp in self.qnalpha:  # go through allowed l,lam combinations

                  # not sure how this prime stuff works here
                  alphap=qnsetp["alpha"]
                  sp = s
                  jp = j
                  j3p = j3
                  tp=t

                  # evaluate wigner 9j symbol
                  c9j = float(Wigner9j(l, s, j, lam, s3, j3, bl, bs, bj).doit())
                  c9jp = float(Wigner9j(l, sp, jp, lam, s3, j3p, bl, bs, bj).doit())

                  # evaluate spin component -> Wigner 6j symnbols
                  c6j_s = float(Wigner6j(s3, 0.5, sp, s3, bs, s).doit())
                  spin_part = (-1)**s * np.sqrt((2*s + 1) * (2*sp + 1)) * c6j_s

                  # include isospin
                  c6j_t = float(Wigner6j(t3, 0.5, tp, t3, bt, t).doit())
                  isospin_part = (-1)**t * np.sqrt((2*t + 1) * (2*tp + 1)) * c6j_t

                  # evaluate coefficients before that 
                  coeff = (2*bs + 1) * np.sqrt((2*j+1) * (2*jp+1) * (2*j3+1) + (2*j3p + 1))

                  for bm in range(-bl,bl+1):
                      if(abs(bm)<=l):  
                          lmindx=self._lmindx(l,bm) 
                          orbital_part = 8*m.pi**2*np.sqrt((2*lam+1)/(4*m.pi))/(2*bl+1) \
                              *ystarl[lmindx,s,j,t,:,:,:]*ylylam[alphap,bm+bl,:,:,:]
          #                 print(bm+bl)
          #                 print(np.nonzero(ylylam[alphap,bm+bl,:,:,:]))

                          gfunc_bl[alpha, alphap, bl, :,:,:] += coeff * c9j * c9jp * spin_part * isospin_part * orbital_part

        gfunc = np.sum(gfunc_bl, axis=2)  # sum over all L, s combinations

        
        self.timegcalc+=timeit.default_timer()
        #  now we assume that there is a function on p on the left defined by p**l and on the right devided by p'**l' 
        # that is interpolated using Cubherm to pi and pip 
        
        # # set spline elements based on grid points and shifted momenta 
        # splpi=Cubherm.spl(self.pgrid,pi)
        # splpip=Cubherm.spl(self.pgrid,pip)
        
        # # interpolation fspl=np.sum(spl*fold,axis=1) first axis is pgrid 
        # # prepare splines multiplied by p**l factors (splalpha also includes the integration weights for q' and x integral)
        
        # splalpha=np.empty((self.npoints*self.nqpoints*self.nalpha,self.nqpoints,self.nx),dtype=np.double)
        # splalphap=np.empty((self.npoints*self.nqpoints*self.nalpha,self.nqpoints,self.nx),dtype=np.double)
        
        # for qnset in self.qnalpha:  # go through allowed l,lam combinations
        #   alpha=qnset["alpha"]
        #   l=qnset["l"]
        #   for ip in range(self.npoints): 
        #    for iq in range(self.nqpoints):
        #      indxpmat=self.npoints*self.nqpoints*alpha+self.npoints*iq+ip
        #      #for jq in range(self.nqpoints):
        #      #   splalpha[indxpmat,jq,:]=splpi[ip,iq,jq,:]*(pi[iq,jq,:]/self.pgrid[ip])**l*self.xw[:]*self.qweight[jq]*self.qgrid[jq]**2
        #      #   splalphap[indxpmat,jq,:]=splpip[ip,jq,iq,:]*(pip[jq,iq,:]/self.pgrid[ip])**l
        #      splalpha[indxpmat,:,:]=np.einsum("ij,j,i->ij",splpi[ip,iq,:,:]*(pi[iq,:,:]/self.pgrid[ip])**l,self.xw[:],self.qweight[:]*self.qgrid[:]**2)
        #      splalphap[indxpmat,:,:]=splpip[ip,:,iq,:]*(pip[:,iq,:]/self.pgrid[ip])**l
            
        
        # pmat=np.empty((self.npoints*self.nqpoints*self.nalpha,self.npoints*self.nqpoints*self.nalpha),dtype=np.double)

        # for qnset in self.qnalpha:  # go through allowed l,lam combinations
        #   alpha=qnset["alpha"]
        #   for qnsetp in self.qnalpha:  # go through allowed l,lam combinations
        #     alphap=qnsetp["alpha"]
        #     for ip in range(self.npoints): 
        #      for iq in range(self.nqpoints):
        #       indxpmat=self.npoints*self.nqpoints*alpha+self.npoints*iq+ip
        #       for jp in range(self.npoints): 
        #        for jq in range(self.nqpoints):
        #         indxpmatp=self.npoints*self.nqpoints*alphap+self.npoints*jq+jp
        #         pmat[indxpmat,indxpmatp]=np.sum(splalpha[indxpmat,jq,:]
        #                       *gfunc[alpha,alphap,iq,jq,:]
        #                       *splalphap[indxpmatp,iq,:]) 
        #
        # set spline elements based on grid points and shifted momenta 
        self.timespl-=timeit.default_timer()
        splpi=Cubherm.spl(self.pgrid,pi)
        splpip=Cubherm.spl(self.pgrid,pip)
        
        # interpolation fspl=np.sum(spl*fold,axis=1) first axis is pgrid 
        # prepare splines multiplied by p**l factors (splalpha also includes the integration weights for q' and x integral)
        
        splalpha=np.empty((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx),dtype=np.double)
        splalphap=np.empty((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx),dtype=np.double)
        
        for qnset in self.qnalpha:  # go through allowed l,lam combinations
          alpha=qnset["alpha"]
          l=qnset["l"]
            
          # ijkl : iq ip jq ix  
          splalpha[alpha,:,:,:,:]=np.einsum("jikl,ikl,j,l,k->ijkl",splpi,pi**l,1.0/self.pgrid**l,self.xw,self.qweight*self.qgrid**2)
          splalphap[alpha,:,:,:,:]=np.einsum("jkil,kil,j->ijkl",splpip,pip**l,1.0/self.pgrid**l)
        
#          for ip in range(self.npoints): 
#           for iq in range(self.nqpoints):
#             indxpmat=self.npoints*self.nqpoints*alpha+self.npoints*iq+ip
             #for jq in range(self.nqpoints):
             #   splalpha[indxpmat,jq,:]=splpi[ip,iq,jq,:]*(pi[iq,jq,:]/self.pgrid[ip])**l*self.xw[:]*self.qweight[jq]*self.qgrid[jq]**2
             #   splalphap[indxpmat,jq,:]=splpip[ip,jq,iq,:]*(pip[jq,iq,:]/self.pgrid[ip])**l
#             splalpha[indxpmat,:,:]=np.einsum("ij,j,i->ij",splpi[ip,iq,:,:]*(pi[iq,:,:]/self.pgrid[ip])**l,self.xw[:],self.qweight[:]*self.qgrid[:]**2)
#             splalphap[indxpmat,:,:]=splpip[ip,:,iq,:]*(pip[:,iq,:]/self.pgrid[ip])**l
        self.timespl+=timeit.default_timer()
            
        
        self.timepmat-=timeit.default_timer()
        #pmat=np.zeros((self.npoints*self.nqpoints*self.nalpha,self.npoints*self.nqpoints*self.nalpha),dtype=np.double)
        
        # also generate views with separated indices 
        #pmatsingle=pmat.reshape((self.nalpha,self.nqpoints,self.npoints,self.nalpha,self.nqpoints,self.npoints))
        
        #splalphapsingle=splalphap.reshape((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx))
        #splalphasingle=splalpha.reshape((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx))
        
        # ijk : alpha iq ip (indxpmat)
        # lmn : alphap jq jp (indxpmatp)
        # o   : ix 
        
        pmatsingle=np.einsum("ijkmo,iljmo,lmnjo->ijklmn",splalpha,gfunc,splalphap)
        pmat=pmatsingle.reshape((self.npoints*self.nqpoints*self.nalpha,self.npoints*self.nqpoints*self.nalpha))
        
#        for qnset in self.qnalpha:  # go through allowed l,lam combinations
#          alpha=qnset["alpha"]
#          for qnsetp in self.qnalpha:  # go through allowed l,lam combinations
#            alphap=qnsetp["alpha"]
#            for ip in range(self.npoints): 
#             for iq in range(self.nqpoints):
#              indxpmat=self.npoints*self.nqpoints*alpha+self.npoints*iq+ip
#              for jp in range(self.npoints): 
#               for jq in range(self.nqpoints):
#                indxpmatp=self.npoints*self.nqpoints*alphap+self.npoints*jq+jp
#                pmat[indxpmat,indxpmatp]=np.sum(splalpha[indxpmat,jq,:]
#                              *gfunc[alpha,alphap,iq,jq,:]
#                              *splalphap[indxpmatp,iq,:])                   
        self.timepmat+=timeit.default_timer()                  
                                      
        return pmat
        
    def prep_faddeev(self,ener):
        """Prepares the Faddeev kernel as a matrix using only two-body interactions.
        
           Parameter:
           ener -- three-body energy in fm-1
        """
 
        # get tmatrix for given energy
        self.tmat=self.prep_tmat(ener)
        
        self.fadpreptime-=timeit.default_timer()
        
        # use matrix multiplication of preprepared permutation matrix 
        # self.pmat[indxpmat,indxpmatp] contains permutation matrix 
        # indexpmat is alpha,iq,ip 

        self.kfadmat=np.zeros(self.pmat.shape,dtype=np.double)
        
        for qnset in self.qnalpha:
            alpha=qnset["alpha"]
            for iq in range(self.nqpoints):
                for ip in range(self.npoints):
                  indxkmat=ip+self.npoints*iq+self.npoints*self.nqpoints*alpha
                  for jp in range(self.npoints):
                    indxpmat=jp+self.npoints*iq+self.npoints*self.nqpoints*alpha
                    self.kfadmat[indxkmat,:]+=self.tmat[alpha,iq,ip,jp]*2*self.pmat[indxpmat,:]
    
        # now multiply with G0
        
        G0=np.empty((self.nqpoints,self.npoints),dtype=np.double)
        for iq in range(self.nqpoints):
          for ip in range(self.npoints):
            G0[iq,ip]=1.0/(ener-0.75*self.qgrid[iq]**2/self.mass-self.pgrid[ip]**2/self.mass )
        
        for alpha in range(self.nalpha):
          for iq in range(self.nqpoints):
            for ip in range(self.npoints):
              indxkmat=ip+self.npoints*iq+self.npoints*self.nqpoints*alpha
              self.kfadmat[indxkmat,:]*=G0[iq,ip]  

        self.fadpreptime+=timeit.default_timer()

# set up set of equations and calculate eigenvalues 

    def eigv(self,E,neigv):
      """Solve three-body Faddev equation and return n-th eigenvalue and Faddeev component. 

         Parameters:
         E -- energy used in the integral equation in fm**-1 
         neigv -- number of the eigenvalue to be used"""
   
    # set up the matrix for the Faddeev equations
      self.prep_faddeev(E)
      self.fadsolvetime-=timeit.default_timer()
        
    # determine eigenvalues using numpy's eig method    
      start=timeit.default_timer()    
      evalue,evec=np.linalg.eig(self.kfadmat)
      print("evalue evaluation: ", timeit.default_timer() - start)
    
    # I now assume that the relevant eigenvalues are real to avoid complex arithmetic 
      evalue=np.real(evalue)
        
    # remove neigv-1 largest eigenvalues 
      for n in range(neigv-1):
        maxpos=np.argmax(evalue)
        evalue[maxpos]=0.0
    
    # take the next one 
      maxpos=np.argmax(evalue)
      eigv=evalue[maxpos]
    
    # define solution as unnormalized Faddeev component 
      fadcomp=np.real(evec[:,maxpos])
          
    # and normalize using permutation again 
      start=timeit.default_timer()    
      fadtmp=2.0*self.pmat.dot(fadcomp)
        
      norm=0.0  
      for alpha in range(self.nalpha):
        for iq in range(self.nqpoints):
          for ip in range(self.npoints):
            indxkmat=ip+self.npoints*iq+self.npoints*self.nqpoints*alpha
            norm+=fadcomp[indxkmat]*fadtmp[indxkmat]*self.qweight[iq]*self.qgrid[iq]**2
            
      fadcomp=fadcomp.reshape((self.nalpha,self.nqpoints,self.npoints))     
      norm+=self.skalp(fadcomp,fadcomp)  
      norm*=3.0
      print("norm evaluation: ", timeit.default_timer() - start)
        
      fadcomp=(1/np.sqrt(norm))*fadcomp
    
      self.fadsolvetime+=timeit.default_timer()
      return eigv,fadcomp
                
    def esearch(self,neigv=1,e1=-0.05,e2=-0.06,elow=-0.02,tol=1e-8,nitermax=20):
        """Perform search for energy using the secant method. 
        
           Parameters:
           neigv -- number of the eigenvalue to be used
           e1 -- first estimate of binding energy (should be negative)
           e2 -- second estimate of binding energy (should be negative)
           elow -- largest energy to be used in search (should be negative)
           tol -- if two consecutive energies differ by less then tol, the search is converged
           
           Energies are given in fm**-1. """
        
        # determine eigenvalues for starting energies        
        eta1,fadcomp=self.eigv(e1,neigv)
        eta2,fadcomp=self.eigv(e2,neigv)
        niter=0
        
        start = timeit.default_timer()
        while abs(e1-e2) > tol: 
          # get new estimate (taking upper value into account)   
          enew=e2+(e1-e2)/(eta1-eta2)*(1-eta2) 
          enew=min(elow,enew)
       
          # get new eigenvalue and replace e1 and e2 for next iteration
          eta,fadcomp=self.eigv(enew,neigv)
          e2=e1
          eta2=eta1
          e1=enew
          eta1=eta 

          print(e1, e2, eta)

          # break if loop is taking too long
          niter+=1
          if niter > nitermax:
            break

        end=timeit.default_timer()

        print("time for energy search: ", end-start)
            
        return e1,eta1,fadcomp 



    # define the wavefunction
    def wavefunc(self, fadcomp):
      '''
      Wavefunction, evaluated from permutation operator and Faddeev component
      '''

      # first evaluate the identity part of the wave function evaluation
      # this means to evaluate the spline integrals without the permutation operator

      # prepare shifted momenta and angles for the symmetric permutation 
      pip=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)        
      pi=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)        
      
      thetap=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)
      theta=np.empty((self.nqpoints,self.nqpoints,self.nx),dtype=np.double)
      thetapp=np.empty((self.nx),dtype=np.double)
      
      for ix in range(self.nx):
        xval=self.xp[ix] 
        thetapp[ix]=np.arccos(xval)
        for jq in range(self.nqpoints):
          qpval=self.qgrid[jq]
          for iq in range(self.nqpoints):
            qval=self.qgrid[iq]
          
            px=qpval*np.sqrt(1.0-xval**2)
            py=0.0
            pz=0.5*qval+qpval*xval 
            pi[iq,jq,ix],theta[iq,jq,ix],phi=self._angle(px,py,pz)
              
            px=-0.5*qpval*np.sqrt(1.0-xval**2)
            py=0.0
            pz=-qval-0.5*qpval*xval 
            pip[iq,jq,ix],thetap[iq,jq,ix],phi=self._angle(px,py,pz)

      splpi=Cubherm.spl(self.pgrid,pi)
      splpip=Cubherm.spl(self.pgrid,pip)
      
      # interpolation fspl=np.sum(spl*fold,axis=1) first axis is pgrid 
      # prepare splines multiplied by p**l factors (splalpha also includes the integration weights for q' and x integral)
      
      splalpha=np.empty((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx),dtype=np.double)
      splalphap=np.empty((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx),dtype=np.double)
      
      for qnset in self.qnalpha:  # go through allowed l,lam combinations
        alpha=qnset["alpha"]
        l=qnset["l"]
          
        # ijkl : iq ip jq ix  
        # we dont integrate over qj so we remove qweights from splalpha
        splalpha[alpha,:,:,:,:]=np.einsum("jikl,ikl,j,l->ijkl",splpi,pi**l,1.0/self.pgrid**l,self.xw)
        splalphap[alpha,:,:,:,:]=np.einsum("jkil,kil,j->ijkl",splpip,pip**l,1.0/self.pgrid**l)
      
        # for ip in range(self.npoints): 
        #   for iq in range(self.nqpoints):
        #     indxpmat=self.npoints*self.nqpoints*alpha+self.npoints*iq+ip
        #     for jq in range(self.nqpoints):
        #       # splalpha[indxpmat,jq,:]=splpi[ip,iq,jq,:]*(pi[iq,jq,:]/self.pgrid[ip])**l*self.xw[:]*self.qweight[jq]*self.qgrid[jq]**2
        #       # splalphap[indxpmat,jq,:]=splpip[ip,jq,iq,:]*(pip[jq,iq,:]/self.pgrid[ip])**l
        #     splalpha[indxpmat,:,:]=np.einsum("ij,j->ij",splpi[ip,iq,:,:]*(pi[iq,:,:]/self.pgrid[ip])**l,self.xw[:])
        #     splalphap[indxpmat,:,:]=splpip[ip,:,iq,:]*(pip[:,iq,:]/self.pgrid[ip])**l

      # idmat = np.zeros((self.npoints*self.nqpoints*self.nalpha,self.npoints*self.nqpoints*self.nalpha), dtype=np.double)

      # constructing the identity matrix element between the spline functions
      idmatsingle=np.einsum("ijkmo,lmnjo->ijklmn",splalpha,splalphap)
      idmat=idmatsingle.reshape((self.npoints*self.nqpoints*self.nalpha,self.npoints*self.nqpoints*self.nalpha))
      
      # also generate views with separated indices 
      #pmatsingle=pmat.reshape((self.nalpha,self.nqpoints,self.npoints,self.nalpha,self.nqpoints,self.npoints))
      
      #splalphapsingle=splalphap.reshape((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx))
      #splalphasingle=splalpha.reshape((self.nalpha,self.nqpoints,self.npoints,self.nqpoints,self.nx))
      
      # ijk : alpha iq ip (indxpmat)
      # lmn : alphap jq jp (indxpmatp)
      # o   : ix 
      
      # pmatsingle=np.einsum("ijkmo,iljmo,lmnjo->ijklmn",splalpha,gfunc,splalphap)
      # pmat=pmatsingle.reshape((self.npoints*self.nqpoints*self.nalpha,self.npoints*self.nqpoints*self.nalpha))

      # for qnset in self.qnalpha:  # go through allowed l,lam combinations
      #   alpha=qnset["alpha"]
      #   for qnsetp in self.qnalpha:  # go through allowed l,lam combinations
      #     alphap=qnsetp["alpha"]
      #     for ip in range(self.npoints): 
      #       for iq in range(self.nqpoints):
      #         indxidmat=self.npoints*self.nqpoints*alpha+self.npoints*iq+ip
      #         for jp in range(self.npoints): 
      #           for jq in range(self.nqpoints):
      #             indxidmatp=self.npoints*self.nqpoints*alphap+self.npoints*jq+jp
      #             idmat[indxidmat,indxidmatp]=np.sum(splalpha[indxidmat,jq,:]
      #                           *splalphap[indxidmatp,iq,:])    

      # now get the wave function

      wf = np.zeros((self.nalpha*self.nqpoints*self.npoints))
      # first reshape faddeev so that it is in terms of one index only
      fadcomp=fadcomp.reshape((self.nalpha*self.nqpoints*self.npoints))   

      # now perform summation over one index
      wf = np.einsum("ij,j->i", idmat + alpha * self.pmat, fadcomp)
      print(wf.shape)

      wf = wf.reshape((self.nalpha,self.nqpoints,self.npoints))

      return wf
    
    
    # the following methods are only useful for testing at this point and can be 
    # ignored at first reading
    
    # application of the Faddeev kernel for the iterative solver 
    
    def faddeev(self,psiin):
        """Applies the Faddeev kernel using only two-body interactions.
             
           Parameter:
           psiin -- incoming Faddeev component

           It assumed that the tmat and G0 have been prepared for the given energy 
           before. 
           
        """
        
        # use matrix multiplication and prepared permutation matrix
        psitmp=2.0*self.pmat.dot(psiin.reshape(-1)).reshape(psiin.shape)
        
        psiout=np.empty(psiin.shape)
        
        for qnset in self.qnalpha:
          l=qnset["l"] 
          alpha=qnset["alpha"]
          psiout[alpha,:,:]=np.einsum("ijk,ik->ij",self.tmat[l,:,:,:],psitmp[alpha,:,:])  
          #for iq in range(self.nqpoints):
          #  for ip in range(self.npoints):
          #    psiout[alpha,iq,ip]=np.sum(self.tmat[l,iq,ip,:]*psitmp[alpha,iq,:])
        
        # now multiply with G0        
        for alpha in range(self.nalpha):
          psiout[alpha,:,:]=psiout[alpha,:,:]*self.G0  
        
        return psiout
        
    
    # the following routines are useful for testing the code. 
    
    def skalp(self,psi1,psi2):
        """Calculate scalar product of two wave functions."""
        
        # multiply with integation weights 
        psitmp=np.zeros((self.nalpha,self.nqpoints,self.npoints),dtype=np.double)        
        for alpha in range(self.nalpha):
         for iq in range(self.nqpoints):
          for ip in range(self.npoints):
           psitmp[alpha,iq,ip]=psi2[alpha,iq,ip] \
                                   *self.pweight[ip]*self.pgrid[ip]**2  \
                                   *self.qweight[iq]*self.qgrid[iq]**2
        return np.sum(psi1*psitmp)
    
    def testperm(self,psi1,psi2):
        """Test permutation matrix."""
        
        # first scalar product without permutation 
        product=self.skalp(psi1,psi2)
        
        # now apply permutation 
        psitmp=self.pmat.dot(psi2.reshape(-1)).reshape(psi2.shape)

        # and build the scalar product (only q integration is necessary)
        for iq in range(self.nqpoints):
           psitmp[:,iq,:] = psitmp[:,iq,:] * self.qgrid[iq]**2*self.qweight[iq]
        
        permprod=np.sum(psitmp*psi1)
    
        print("Permutation test:  {0:15.6e}   {1:15.6e}".format(product,permprod))
    
    def testfu(self):
        """Prepares a fully symmetrical wave function."""
        
        psitmp=np.zeros((self.nalpha,self.nqpoints,self.npoints),dtype=np.double)
        for qnset in self.qnalpha:
         if qnset["l"]==0 and qnset["lam"]==0:  
           alpha=qnset["alpha"] 
           for iq in range(self.nqpoints):
            for ip in range(self.npoints):
             x=self.pgrid[ip]**2 +0.75*self.qgrid[iq]**2      
             psitmp[alpha,iq,ip]=np.exp(-0.05*x)/(0.05*x**2+10.0)
                
                
        return psitmp
    
    def printwf(self,psi):
        """Prints wave function."""
     
        psitest=psi.reshape(-1)        
        for j in range(self.npoints*self.nqpoints*self.nalpha):    
            alphap=j//(self.npoints*self.nqpoints)
            jq=(j-alphap*self.npoints*self.nqpoints)//self.npoints
            jp=(j-self.npoints*self.nqpoints*alphap-self.npoints*jq)#

            print("{0:s}   {1:4d}    {2:4d} {3:4d} {4:4d}       {5:15.6e}  {6:15.6e}".format("testfu",j,jp,jq,alphap,psitest[j],psi[alphap,jq,jp]))
    
   
    def comparewf(self,psi1,psi2,tolrel,tolabs):
        """Compares two wave functions."""
     
        for j in range(self.npoints*self.nqpoints*self.nalpha):    
            alphap=j//(self.npoints*self.nqpoints)
            jq=(j-alphap*self.npoints*self.nqpoints)//self.npoints
            jp=(j-self.npoints*self.nqpoints*alphap-self.npoints*jq)
            
            if abs(psi1[alphap,jq,jp]-psi2[alphap,jq,jp])>tolabs \
               or abs(psi1[alphap,jq,jp]-psi2[alphap,jq,jp])/max(abs(psi1[alphap,jq,jp]),tolabs) > tolrel:
            
              print("{0:s}   {1:4d}    {2:4d} {3:4d} {4:4d}       {5:15.6e}  {6:15.6e}    {7:15.6e}".format("Compare:",j,jp,jq,alphap,
                                        psi1[alphap,jq,jp],psi2[alphap,jq,jp],(psi1[alphap,jq,jp]-psi2[alphap,jq,jp])/psi1[alphap,jq,jp]))

   
    def eigv_iter(self,E,neigv, maxiter=10, eigtol=1e-6):
        """Solve three-body Faddev equation and return n-th eigenvalue and Faddeev component. 

         Parameters:
         E -- energy used in the integral equation in fm**-1 
         neigv -- number of the eigenvalue to be used"""

        self.fadsolvetime-=timeit.default_timer()
    
        # get tmatrix for given energy
        self.tmat=self.prep_tmat(E)
        # calculate G0 for E 
        self.G0=self.prep_G0(E) 
        
        # define a start vector of a constant value in each component
        # and normalize   
        psistart=np.ones((self.nalpha,self.nqpoints,self.npoints),dtype=np.double)
        norm=self.skalp(psistart,psistart)
        psistart=(1/np.sqrt(norm))*psistart
        
        # define array for basis vectors, first one is start vector 
        psiv=np.empty((maxiter+1,self.nalpha,self.nqpoints,self.npoints),dtype=np.double)
        psiv[0,:,:,:]=psistart[:,:,:]
        
        # define array for < v_i | K | v_j > 
        bmat=np.zeros((maxiter+1,maxiter+1),dtype=np.double)
        # for comparison to check convergence 
        lasteta=0.0   
        
        for n in range(maxiter):  # start iteration 
          # apply kernel   
          psiw=self.faddeev(psiv[n,:,:,:])
          # count iterations for stastics 
        
          self.numiter+=1  
          # orthogonalize 
          psitildew=np.empty(psiw.shape,dtype=np.double)
          psitildew[:,:,:]=psiw[:,:,:] 
        
          for k in range(n+1):
            skaprod=self.skalp(psiv[k,:,:,:],psiw)    
            psitildew-=skaprod*psiv[k,:,:,:]
            # keep relevant matrix elements in bmat 
            bmat[k,n]=skaprod
          
          # now normalize the orthogonal wtilde and store new basis vector 
          skaprod=self.skalp(psitildew,psitildew)
          psitildew=(1/np.sqrt(skaprod))*psitildew 
          psiv[n+1,:,:,:]=psitildew[:,:,:]
          # finallay store last new element of bmat           
          bmat[n+1,n]=np.sqrt(skaprod)  
          
          # in each step eigenvalues of bmat can be calculated and compared to previous eigenvalues
          # determine eigenvalues using numpy's eig method (only use dimensions already defined)
          evalue,evec=np.linalg.eig(bmat[0:n+1,0:n+1])
    
          # I now assume that the relevant eigenvalues are real to avoid complex arithmetic 
          evalue=np.real(evalue)
        
          # remove neigv-1 largest eigenvalues 
          for n in range(neigv-1):
            maxpos=np.argmax(evalue)
            evalue[maxpos]=0.0
    
          # and take the next one 
          maxpos=np.argmax(evalue)
          eigv=evalue[maxpos]
          if (np.abs(lasteta-eigv)<eigtol): # converged, stop iteration 
            break 
            
          lasteta=eigv   
        
        # now we assume a converged eigenvalue
        # use the corresponding eigenvector to obtaine the Faddeev component
        fadcomp=np.einsum("l,lijk->ijk",np.real(evec[:,maxpos]),psiv[0:n+1,:,:,:])
        self.fadsolvetime+=timeit.default_timer()

        return eigv,fadcomp   
# preparation for G0 for a given energy 

    def prep_G0(self,E):
      """Prepares G0 for a given energy.
      
          E -- three-body energy in fm**-1 
          
          returns G0 
      """  
    
      G0=np.empty((self.nqpoints,self.npoints),dtype=np.double)  
      for iq in range(self.nqpoints):
        for ip in range(self.npoints):
          G0[iq,ip]=1.0/(E-0.75*self.qgrid[iq]**2/self.mass-self.pgrid[ip]**2/self.mass)
        
      return G0  

In [8]:
para=[700.0, 0.020167185806378923]

pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])
kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0, l3max=0, bj=0.5)
eigv, fadcomp = kernel.eigv_iter(E=-0.004713, neigv=1)

Duration of alpha list construction:  0.0007914999999911743


In [9]:
para=[700.0, 0.020167185806378923]

pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])
kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0, l3max=0, bj=0.5)
# ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.05,e2=-0.06,elow=-0.02,tol=1e-8)
# ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0007,e2=-0.0009,elow=-1e-5,tol=1e-1)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)

# the magic numbers: -0.0046, -0.0048, elow=-0.001, tol=1e-5

print("CG times: ", kernel.timeclebsch)
print("sph harm times: ", kernel.timeyl, kernel.timeylam, kernel.timeystarl)
print("coupled sph harm time: ", kernel.timeylylam)
print("gfunc time: ", kernel.timegcalc)
print("spline time: ", kernel.timespl)
print("pmat time: ", kernel.timepmat)
print("Faddeev preparing time: ", kernel.fadpreptime)
print("Faddeev solve time: ", kernel.fadsolvetime)
print("tmat prepare time: ", kernel.tmattime)

print("Ener:",ener*kernel.hbarc,eta)

Duration of alpha list construction:  0.00016579999999066786
evalue evaluation:  152.91838620000001
norm evaluation:  0.4170154999999909
evalue evaluation:  151.29761070000006
norm evaluation:  0.4746362000000772
evalue evaluation:  150.17620900000009
norm evaluation:  0.5843861999999262
-0.004706690994937736 -0.0046 0.9944373411500812
evalue evaluation:  148.97062390000008
norm evaluation:  0.4396351999999979
-0.004713817067497682 -0.004706690994937736 1.0004301176206258
time for energy search:  330.46702200000004
CG times:  0.013252500000021428
sph harm times:  0.0019888999999864154 0.002587499999975762 0.001366600000011431
coupled sph harm time:  0.0016753000000164775
gfunc time:  5.010139400000014
spline time:  2.2876997000000188
pmat time:  24.366434099999992
Faddeev preparing time:  55.77455250000014
Faddeev solve time:  605.4859275
tmat prepare time:  2.248048899999958
Ener: -0.9301633804781151 1.0004301176206258


In [10]:
wf = kernel.wavefunc(fadcomp)

(12288,)


We should do this for different grid points so that we get something that converges pretty well.
- need to delete kernel to prevent overload in saving for RAM

In [8]:
para=[700.0, 0.020167185806378923]

pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])
kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)

del kernel

kernel=ThreeBody(pot,nx=12,np1=20,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)

del kernel

kernel=ThreeBody(pot,nx=16,np1=20,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)

del kernel

kernel=ThreeBody(pot,nx=8,np1=16,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)

del kernel

kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)

del kernel

kernel=ThreeBody(pot,nx=8,np1=24,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)
del kernel

kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=16,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)

del kernel

kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)

del kernel

kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=24,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
print("Ener:",ener*kernel.hbarc,eta)


Duration of alpha list construction:  5.920000000969594e-05
Ener: -3.94654 0.09976876097156828
Duration of alpha list construction:  0.0008149999999886859
Ener: -3.94654 0.09977795721324609
Duration of alpha list construction:  0.00012590000005729962
Ener: -3.94654 0.09977464440970818


TypeError: __init__() got an unexpected keyword argument 'bl'

Now we need to evaluate the wavefunction by using the permutation operator and the faddeev equation

In [ ]:
# check the performance of each component to see where it takes the longest to compute
para=[700.0, 0.020167185806378923]

# start = timeit.default_timer()
pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])
kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0, jmax=1, l3max=0, bj=0.5)
# ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.05,e2=-0.06,elow=-0.02,tol=1e-8)
# ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0001,e2=-0.01,elow=-1e-5,tol=1e-6)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0001,e2=-0.01,elow=-1e-5,tol=1e-2)
# end = timeit.default_timer()

# evaluate the wavefunction
wf = kernel.wavefunc(fadcomp)

